# Benchmarks - Hyper-parameters optimisation

**Main considerations when implementing HPs optimisation**
- we made kernels pytrees, so we should be able to compute gradient and optimise for them directly


---
## Setup

In [1]:
# Standard library
import os
from typing import NamedTuple

os.environ['JAX_ENABLE_X64'] = "True"

In [2]:
# Third party
import jax
import jax.numpy as jnp
import jax.random as jrd
from jax.tree_util import tree_flatten
import optax
import optax.tree_utils as otu
import chex

import numpy as np
import pandas as pd

In [3]:
# Local
from Kernax import SEMagmaKernel, DiagKernel, ExpKernel
from MagmaClustPy.utils import preprocess_db
from MagmaClustPy.hyperpost import hyperpost

In [4]:
# Config
key = jrd.PRNGKey(0)
test_db_size = "small"

---
## Data

---
## Current implementation

In [5]:
import MagmaClustPy
optimise_hyperparameters_old = MagmaClustPy.hp_optimisation.optimise_hyperparameters

---
## Custom implementation(s)

In [6]:
from typing import NamedTuple

import chex
import jax
import jax.numpy as jnp
import optax
import optax.tree_utils as otu

from MagmaClustPy.likelihoods import magma_neg_likelihood


# Taken from optax doc (https://optax.readthedocs.io/en/latest/_collections/examples/lbfgs.html#l-bfgs-solver)
class InfoState(NamedTuple):
	iter_num: chex.Numeric


def print_info():
	def init_fn(params):
		del params
		return InfoState(iter_num=0)

	def update_fn(updates, state, params, *, value, grad, **extra_args):
		del params, extra_args

		jax.debug.print(
			'Iteration: {i}, Value: {v}, Gradient norm: {e}',
			i=state.iter_num,
			v=value,
			e=otu.tree_norm(grad),
		)
		return updates, InfoState(iter_num=state.iter_num + 1)

	return optax.GradientTransformationExtraArgs(init_fn, update_fn)


# Adapted from optax doc (https://optax.readthedocs.io/en/latest/_collections/examples/lbfgs.html#l-bfgs-solver)
def run_opt(init_params, fun, opt, max_iter, tol):
	value_and_grad_fun = optax.value_and_grad_from_state(fun)

	def step(carry):
		params, state, prev_llh = carry
		value, grad = value_and_grad_fun(params, state=state)
		updates, state = opt.update(grad, state, params, value=value, grad=grad, value_fn=fun)
		params = optax.apply_updates(params, updates)
		return params, state, value

	def continuing_criterion(carry):
		# tol is not computed on the gradients but on the difference between current and previous likelihoods, to
		# prevent overfitting on ill-defined likelihood functions where variance can blow up.
		_, state, prev_llh = carry
		iter_num = otu.tree_get(state, 'count')
		val = otu.tree_get(state, 'value')
		diff = jnp.abs(val - prev_llh)
		return (iter_num == 0) | ((iter_num < max_iter) & (diff >= tol))

	init_carry = (init_params, opt.init(init_params),
	              jnp.array(jnp.inf))  # kernel params, initial state, first iter, previous likelihood
	final_params, final_state, final_llh = jax.lax.while_loop(
		continuing_criterion, step, init_carry
	)
	return final_params, final_state, final_llh


def optimise_hyperparameters(mean_kernel, task_kernel, inputs, outputs, all_inputs, prior_mean, post_mean, post_cov,
                             mappings, nugget=jnp.array(1e-10), max_iter=100, tol=1e-3, verbose=False):
	# Optimise mean kernel
	if verbose:
		mean_opt = optax.chain(print_info(), optax.lbfgs())
	else:
		mean_opt = optax.lbfgs()

	def mean_fun_wrapper(kern):
		res = magma_neg_likelihood(kern, all_inputs, post_mean, prior_mean, post_cov, None, nugget=nugget)
		return res

	new_mean_kernel, _, mean_llh = run_opt(mean_kernel, mean_fun_wrapper, mean_opt, max_iter=max_iter, tol=tol)

	# Optimise task kernel
	if verbose:
		task_opt = optax.chain(print_info(), optax.lbfgs())
	else:
		task_opt = optax.lbfgs()

	def task_fun_wrapper(kern):
		res = magma_neg_likelihood(kern, inputs, outputs, post_mean, post_cov, mappings, nugget=nugget).sum()
		return res

	new_task_kernel, _, task_llh = run_opt(task_kernel, task_fun_wrapper, task_opt, max_iter=max_iter, tol=tol)

	return new_mean_kernel, new_task_kernel, mean_llh, task_llh

In [7]:
optimise_hyperparameters_new = optimise_hyperparameters

---
## Comparison

In [8]:
nugget = jnp.array(1e-10)

### shared Input, shared HP

In [9]:
db = pd.read_csv(f"../datasets/{test_db_size}_shared_input_shared_hp.csv")
padded_inputs, padded_outputs, masks, all_inputs = preprocess_db(db)
prior_mean = jnp.array(0)
all_inputs.shape, padded_inputs.shape

((15, 1), (20, 15, 1))

In [10]:
mean_kern = SEMagmaKernel(length_scale=.3, variance=1.)
task_kern = SEMagmaKernel(length_scale=.3, variance=1.) + DiagKernel(ExpKernel(2.5))

In [11]:
post_mean, post_cov = hyperpost(padded_inputs, padded_outputs, masks, prior_mean, mean_kern, task_kern, all_inputs=all_inputs, nugget=nugget)
post_mean.shape, post_cov.shape

((15,), (15, 15))

In [12]:
optimized_mean_kern_old, optimized_task_kern_old, _, _ = optimise_hyperparameters_old(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget, verbose=True)

AttributeError: 'float' object has no attribute 'shape'

In [13]:
optimized_mean_kern_old, optimized_task_kern_old

(SEMagmaKernel(length_scale=1.1709656865266187, variance=5.843095076461786),
 SumKernel(left_kernel=SEMagmaKernel(length_scale=1.6343234360292214, variance=3.336620286865063), right_kernel=DiagKernel(inner_kernel=ExpKernel(inner_kernel=ConstantKernel(value=0.7728861273444985)))))

In [14]:
optimized_mean_kern_new, optimized_task_kern_new, _, _ = optimise_hyperparameters_new(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget, verbose=True)

Iteration: 0, Value: 836.6045316967227, Gradient norm: 842.3511548986307
Iteration: 1, Value: 314.1133811574371, Gradient norm: 295.11474538671615
Iteration: 2, Value: 192.89621304539364, Gradient norm: 166.17834784251815
Iteration: 3, Value: 111.14324986746566, Gradient norm: 77.97054924188863
Iteration: 4, Value: 76.1837911272774, Gradient norm: 38.653273391248305
Iteration: 5, Value: 59.56567545948463, Gradient norm: 18.124596262951094
Iteration: 6, Value: 52.74446277509758, Gradient norm: 8.265713962882861
Iteration: 7, Value: 50.57056501236482, Gradient norm: 9.332184246722381
Iteration: 8, Value: 49.94586723766478, Gradient norm: 6.403268816933967
Iteration: 9, Value: 49.62739052343301, Gradient norm: 2.848933628157671
Iteration: 10, Value: 49.5836933613408, Gradient norm: 1.539941647377355
Iteration: 11, Value: 49.57023956387713, Gradient norm: 0.2128016972688677
Iteration: 0, Value: 888.4353418332946, Gradient norm: 44.384857897924874
Iteration: 1, Value: 882.7911771996015, Gra

In [15]:
optimized_mean_kern_new, optimized_task_kern_new

(SEMagmaKernel(length_scale=1.1709656865266187, variance=5.843095076461786),
 SumKernel(left_kernel=SEMagmaKernel(length_scale=1.6343234360292214, variance=3.336620286865063), right_kernel=DiagKernel(inner_kernel=ExpKernel(inner_kernel=ConstantKernel(value=0.7728861273444985)))))

In [16]:
%%timeit -n 3 -r 2
optimise_hyperparameters_old(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget)[0].length_scale.block_until_ready()

907 ms ± 18.1 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


In [17]:
%%timeit -n 3 -r 2
optimise_hyperparameters_new(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget)[0].length_scale.block_until_ready()

909 ms ± 4.46 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


### shared Input, Distinct HP

In [18]:
db = pd.read_csv(f"../datasets/{test_db_size}_shared_input_distinct_hp.csv")
padded_inputs, padded_outputs, masks, all_inputs = preprocess_db(db)
prior_mean = jnp.array(0)
all_inputs.shape, padded_inputs.shape

((15, 1), (20, 15, 1))

In [19]:
mean_kern = SEMagmaKernel(length_scale=.3, variance=1.)

key, subkey = jax.random.split(key)
distinct_length_scales = jax.random.uniform(subkey, (padded_outputs.shape[0],), jnp.float64, .1, 1)
task_kern = SEMagmaKernel(length_scale=.3, variance=1.) + DiagKernel(ExpKernel(2.5))

In [20]:
post_mean, post_cov = hyperpost(padded_inputs, padded_outputs, masks, prior_mean, mean_kern, task_kern, all_inputs=all_inputs, nugget=nugget)
post_mean.shape, post_cov.shape

((15,), (15, 15))

In [21]:
optimized_mean_kern_old, optimized_task_kern_old, _, _ = optimise_hyperparameters_old(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget, verbose=True)

Iteration: 0, Value: 1738.9864707607353, Gradient norm: 1856.2079389251314
Iteration: 1, Value: 614.7029917986152, Gradient norm: 613.5748706550256
Iteration: 2, Value: 377.91304943406533, Gradient norm: 356.11966791768685
Iteration: 3, Value: 193.6458111094163, Gradient norm: 166.53767301371266
Iteration: 4, Value: 117.38174596723175, Gradient norm: 83.34248985026642
Iteration: 5, Value: 79.02482465557843, Gradient norm: 40.97287325647887
Iteration: 6, Value: 61.73413500383541, Gradient norm: 19.590938292952515
Iteration: 7, Value: 54.90999862201198, Gradient norm: 14.566858990779826
Iteration: 8, Value: 51.624993523019555, Gradient norm: 4.909153488669724
Iteration: 9, Value: 51.1236104917011, Gradient norm: 7.1686189599739585
Iteration: 10, Value: 50.90517900664831, Gradient norm: 0.7889159550129479
Iteration: 11, Value: 50.8976149597436, Gradient norm: 0.21691130476932338
Iteration: 0, Value: 1018.6425284142251, Gradient norm: 142.96851439071645
Iteration: 1, Value: 952.76939396825

In [22]:
optimized_mean_kern_old, optimized_task_kern_old

(SEMagmaKernel(length_scale=1.1479675743688122, variance=6.555698067746036),
 SumKernel(left_kernel=SEMagmaKernel(length_scale=5.751764264682112, variance=3.6571825489185725), right_kernel=DiagKernel(inner_kernel=ExpKernel(inner_kernel=ConstantKernel(value=2.097379134000261)))))

In [23]:
optimized_mean_kern_new, optimized_task_kern_new, _, _ = optimise_hyperparameters_new(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget, verbose=True)

Iteration: 0, Value: 1738.9864707607353, Gradient norm: 1856.2079389251314
Iteration: 1, Value: 614.7029917986152, Gradient norm: 613.5748706550256
Iteration: 2, Value: 377.91304943406533, Gradient norm: 356.11966791768685
Iteration: 3, Value: 193.6458111094163, Gradient norm: 166.53767301371266
Iteration: 4, Value: 117.38174596723175, Gradient norm: 83.34248985026642
Iteration: 5, Value: 79.02482465557843, Gradient norm: 40.97287325647887
Iteration: 6, Value: 61.73413500383541, Gradient norm: 19.590938292952515
Iteration: 7, Value: 54.90999862201198, Gradient norm: 14.566858990779826
Iteration: 8, Value: 51.624993523019555, Gradient norm: 4.909153488669724
Iteration: 9, Value: 51.1236104917011, Gradient norm: 7.1686189599739585
Iteration: 10, Value: 50.90517900664831, Gradient norm: 0.7889159550129479
Iteration: 11, Value: 50.8976149597436, Gradient norm: 0.21691130476932338
Iteration: 0, Value: 1018.6425284142251, Gradient norm: 142.96851439071645
Iteration: 1, Value: 952.76939396825

In [24]:
optimized_mean_kern_new, optimized_task_kern_new

(SEMagmaKernel(length_scale=1.1479675743688122, variance=6.555698067746036),
 SumKernel(left_kernel=SEMagmaKernel(length_scale=5.751764264682112, variance=3.6571825489185725), right_kernel=DiagKernel(inner_kernel=ExpKernel(inner_kernel=ConstantKernel(value=2.097379134000261)))))

In [25]:
%%timeit -n 3 -r 2
optimise_hyperparameters_old(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget)[0].length_scale.block_until_ready()

910 ms ± 20 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


In [26]:
%%timeit -n 3 -r 2
optimise_hyperparameters_new(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget)[0].length_scale.block_until_ready()

913 ms ± 8.95 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


### Distinct Input, shared HP

In [27]:
db = pd.read_csv(f"../datasets/{test_db_size}_distinct_input_shared_hp.csv")
padded_inputs, padded_outputs, masks, all_inputs = preprocess_db(db)
prior_mean = jnp.array(0)
all_inputs.shape, padded_inputs.shape

((41, 1), (20, 15, 1))

In [28]:
mean_kern = SEMagmaKernel(length_scale=.3, variance=1.)
task_kern = SEMagmaKernel(length_scale=.3, variance=1.) + DiagKernel(ExpKernel(2.5))

In [29]:
post_mean, post_cov = hyperpost(padded_inputs, padded_outputs, masks, prior_mean, mean_kern, task_kern, all_inputs=all_inputs, nugget=nugget)
post_mean.shape, post_cov.shape

((41,), (41, 41))

In [30]:
optimized_mean_kern_old, optimized_task_kern_old, _, _ = optimise_hyperparameters_old(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget, verbose=True)

Iteration: 0, Value: 7632568.6129706325, Gradient norm: 11013938.746285157
Iteration: 1, Value: 2920131.8379903315, Gradient norm: 2926246.5190971177
Iteration: 2, Value: 1952581.1574465851, Gradient norm: 1955369.2266142257
Iteration: 3, Value: 869176.5626329303, Gradient norm: 869709.4220277806
Iteration: 4, Value: 454893.7191824574, Gradient norm: 454974.1652769489
Iteration: 5, Value: 223752.82748765693, Gradient norm: 223686.37303180672
Iteration: 6, Value: 112747.98392579293, Gradient norm: 112636.82285557516
Iteration: 7, Value: 56320.51984895356, Gradient norm: 56188.51684458967
Iteration: 8, Value: 28274.054315999383, Gradient norm: 28126.635247176222
Iteration: 9, Value: 14219.686049294933, Gradient norm: 14057.594493652585
Iteration: 10, Value: 7204.402377523664, Gradient norm: 7027.678691245698
Iteration: 11, Value: 3702.0178927986435, Gradient norm: 3510.780628696635
Iteration: 12, Value: 1957.2671294474103, Gradient norm: 1752.2204017473512
Iteration: 13, Value: 1090.5381

In [31]:
optimized_mean_kern_old, optimized_task_kern_old

(SEMagmaKernel(length_scale=0.8867817010832448, variance=15.783555092593412),
 SumKernel(left_kernel=SEMagmaKernel(length_scale=0.4404868067753267, variance=-0.48698077333676776), right_kernel=DiagKernel(inner_kernel=ExpKernel(inner_kernel=ConstantKernel(value=14.313616111309392)))))

In [32]:
optimized_mean_kern_new, optimized_task_kern_new, _, _ = optimise_hyperparameters_new(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget, verbose=True)

Iteration: 0, Value: 7632568.6129706325, Gradient norm: 11013938.746285157
Iteration: 1, Value: 2920131.8379903315, Gradient norm: 2926246.5190971177
Iteration: 2, Value: 1952581.1574465851, Gradient norm: 1955369.2266142257
Iteration: 3, Value: 869176.5626329303, Gradient norm: 869709.4220277806
Iteration: 4, Value: 454893.7191824574, Gradient norm: 454974.1652769489
Iteration: 5, Value: 223752.82748765693, Gradient norm: 223686.37303180672
Iteration: 6, Value: 112747.98392579293, Gradient norm: 112636.82285557516
Iteration: 7, Value: 56320.51984895356, Gradient norm: 56188.51684458967
Iteration: 8, Value: 28274.054315999383, Gradient norm: 28126.635247176222
Iteration: 9, Value: 14219.686049294933, Gradient norm: 14057.594493652585
Iteration: 10, Value: 7204.402377523664, Gradient norm: 7027.678691245698
Iteration: 11, Value: 3702.0178927986435, Gradient norm: 3510.780628696635
Iteration: 12, Value: 1957.2671294474103, Gradient norm: 1752.2204017473512
Iteration: 13, Value: 1090.5381

In [33]:
optimized_mean_kern_new, optimized_task_kern_new

(SEMagmaKernel(length_scale=0.8867817010832448, variance=15.783555092593412),
 SumKernel(left_kernel=SEMagmaKernel(length_scale=0.4404868067753267, variance=-0.48698077333676776), right_kernel=DiagKernel(inner_kernel=ExpKernel(inner_kernel=ConstantKernel(value=14.313616111309392)))))

In [34]:
%%timeit -n 3 -r 2
optimise_hyperparameters_old(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget)[0].length_scale.block_until_ready()

1.04 s ± 7.94 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


In [35]:
%%timeit -n 3 -r 2
optimise_hyperparameters_new(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget)[0].length_scale.block_until_ready()

1.03 s ± 1.24 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


### Distinct Input, Distinct HP

In [36]:
db = pd.read_csv(f"../datasets/{test_db_size}_distinct_input_distinct_hp.csv")
padded_inputs, padded_outputs, masks, all_inputs = preprocess_db(db)
prior_mean = jnp.array(0)
all_inputs.shape, padded_inputs.shape

((41, 1), (20, 19, 1))

In [37]:
mean_kern = SEMagmaKernel(length_scale=.3, variance=1.)

key, subkey = jax.random.split(key)
distinct_length_scales = jax.random.uniform(subkey, (padded_outputs.shape[0],), jnp.float64, .1, 1)
task_kern = SEMagmaKernel(length_scale=.3, variance=1.) + DiagKernel(ExpKernel(2.5))

In [38]:
post_mean, post_cov = hyperpost(padded_inputs, padded_outputs, masks, prior_mean, mean_kern, task_kern, all_inputs=all_inputs, nugget=nugget)
post_mean.shape, post_cov.shape

((41,), (41, 41))

In [39]:
optimized_mean_kern_old, optimized_task_kern_old, _, _ = optimise_hyperparameters_old(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget, verbose=True)

Iteration: 0, Value: 190467.96517876248, Gradient norm: 364147.9343514594
Iteration: 1, Value: 70687.33262344608, Gradient norm: 71151.35482363835
Iteration: 2, Value: 54587.148943791995, Gradient norm: 54758.82799875409
Iteration: 3, Value: 23238.841829142337, Gradient norm: 23169.996476653152
Iteration: 4, Value: 12518.4407578984, Gradient norm: 12430.065820624288
Iteration: 5, Value: 6175.625672469804, Gradient norm: 6074.888620102092
Iteration: 6, Value: 3187.1238505582687, Gradient norm: 3072.406505160453
Iteration: 7, Value: 1660.8179540650058, Gradient norm: 1530.7027459345343
Iteration: 8, Value: 910.6021865349865, Gradient norm: 765.6355834289982
Iteration: 9, Value: 540.3338495419432, Gradient norm: 381.93900680731895
Iteration: 10, Value: 360.87765266164695, Gradient norm: 191.84876565894726
Iteration: 11, Value: 274.64747957731146, Gradient norm: 100.75762915955656
Iteration: 12, Value: 219.3269928390746, Gradient norm: 71.24163075267964
Iteration: 13, Value: 149.9520115853

In [40]:
optimized_mean_kern_old, optimized_task_kern_old

(SEMagmaKernel(length_scale=0.7288395212051347, variance=11.906479811901379),
 SumKernel(left_kernel=SEMagmaKernel(length_scale=0.18794220954840343, variance=0.4148712219921758), right_kernel=DiagKernel(inner_kernel=ExpKernel(inner_kernel=ConstantKernel(value=10.310210776967464)))))

In [41]:
optimized_mean_kern_new, optimized_task_kern_new, _, _ = optimise_hyperparameters_new(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget, verbose=True)

Iteration: 0, Value: 190467.96517876248, Gradient norm: 364147.9343514594
Iteration: 1, Value: 70687.33262344608, Gradient norm: 71151.35482363835
Iteration: 2, Value: 54587.148943791995, Gradient norm: 54758.82799875409
Iteration: 3, Value: 23238.841829142337, Gradient norm: 23169.996476653152
Iteration: 4, Value: 12518.4407578984, Gradient norm: 12430.065820624288
Iteration: 5, Value: 6175.625672469804, Gradient norm: 6074.888620102092
Iteration: 6, Value: 3187.1238505582687, Gradient norm: 3072.406505160453
Iteration: 7, Value: 1660.8179540650058, Gradient norm: 1530.7027459345343
Iteration: 8, Value: 910.6021865349865, Gradient norm: 765.6355834289982
Iteration: 9, Value: 540.3338495419432, Gradient norm: 381.93900680731895
Iteration: 10, Value: 360.87765266164695, Gradient norm: 191.84876565894726
Iteration: 11, Value: 274.64747957731146, Gradient norm: 100.75762915955656
Iteration: 12, Value: 219.3269928390746, Gradient norm: 71.24163075267964
Iteration: 13, Value: 149.9520115853

In [42]:
optimized_mean_kern_new, optimized_task_kern_new

(SEMagmaKernel(length_scale=0.7288395212051347, variance=11.906479811901379),
 SumKernel(left_kernel=SEMagmaKernel(length_scale=0.18794220954840343, variance=0.4148712219921758), right_kernel=DiagKernel(inner_kernel=ExpKernel(inner_kernel=ConstantKernel(value=10.310210776967464)))))

In [43]:
%%timeit -n 3 -r 2
optimise_hyperparameters_old(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget)[0].length_scale.block_until_ready()

1.04 s ± 1.83 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


In [44]:
%%timeit -n 3 -r 2
optimise_hyperparameters_new(mean_kern, task_kern, padded_inputs, padded_outputs, all_inputs, prior_mean, post_mean, post_cov, masks, nugget=nugget)[0].length_scale.block_until_ready()

1.04 s ± 12.1 ms per loop (mean ± std. dev. of 2 runs, 3 loops each)


---
## Conclusion

---